In this notebook, we debug the `_test_tau_leap_safety` function (old version written in python).
Fixing this will help to understand and fix the Cython version.

First load the relevant modules:

In [1]:
from pygom import SimulateOde, Transition, TransitionType
import numpy as np
import math

Set up PyGOM object. This will be an SIR model with births and deaths:

In [2]:
state = ['S', 'I', 'R']
param_list = ['beta', 'gamma', 'mu', 'N']

transition = [
    Transition(origin='S',
               destination='I',
               equation='beta*S*I/N',
               transition_type=TransitionType.T),
    Transition(origin='I',
               destination='R',
               equation='gamma*I',
               transition_type=TransitionType.T)
]

birth_death = [
    Transition(origin='S',
               equation='mu*N',
               transition_type=TransitionType.B),
    Transition(origin='S',
               equation='mu*S',
               transition_type=TransitionType.D),
    Transition(origin='I',
               equation='mu*I',
               transition_type=TransitionType.D),
    Transition(origin='R',
               equation='mu*R',
               transition_type=TransitionType.D)
]

# initialize the model
ode = SimulateOde(state,
                  param_list,
                  birth_death=birth_death,
                  transition=transition)

#Params
n_pop=1e4
param_set=[('beta', 0.4), ('gamma', 0.25), ('mu', 0.01), ('N', n_pop)]
ode.parameters=param_set

# Initial conditions
i0=10
x0 = [n_pop-i0, i0, 0]
ode.initial_values = (x0, 0)

# Calculate reactant matrix
ode._computeReactantMatrix()

array([[1, 0, 0, 1, 0, 0],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

Define old python version of function here. I've added a few comments in

In [3]:
## Python solution

import scipy.stats as st

def _ppois(q, mu=1.0):
    '''
    A cached and slightly faster and less safe version of the pygom.utilR.ppois
    function
    '''
    return st.poisson._cdf(q, mu=mu)

def _test_tau_leap_safety(x, reactant_mat, rates, tau_scale, epsilon):
    """
    Additional safety test on :math:`\\tau`-leap, decrease the step size if
    the original is not small enough.  Decrease a couple of times and then
    bail out because we don't want to spend too long decreasing the
    step size until we find a suitable one.

    Parameters
    ----------
    x: array
        Current populations of the states
    reactant_mat: matrix
        reactant_mat[i,j]=1 if state i involved in transition j and 0 if not
    rates: array
        rates of each of the transitions
    tau_scale: float
        initial value for timestep
    epsilon: float
        threshold value
    """

    total_rate = sum(rates)  
    safe = False             # when True, indicates that tau_scale is sufficiently small
    count = 0                # number of attempts to find acceptable tau_scale

    # Print output, first some titles
    row=["Iteration", "tau_scale", "max_cdf"]
    print("{: <10} {: <10} {: <10}".format(*row))

    while safe is False:
        cdf_val = 1.0
        for i, r in enumerate(rates):
            xi = x[reactant_mat[:, i]]         # reactant_mat[i,j]={0,1} so we only ever look at first 2 states. Seems wrong
            mu=tau_scale*r                     # Expected number of events
            new_cdf = _ppois(xi, mu=mu).min()  # prob(# transitions in jump < size of state)
            if new_cdf < cdf_val:
                cdf_val = new_cdf

        max_cdf = 1.0 - cdf_val  # prob(# transmissions in jump > size of state)

        # Print output
        row=[count, tau_scale, max_cdf]
        print("{: <10} {: <10.4g} {: <10.4g}".format(*row))

        # cannot allow it to exceed out epsilon
        if max_cdf > epsilon:
            tau_scale /= (max_cdf / epsilon)
        else:
            safe = True

        # Abandon if we're taking too many attempts
        if count > 256:
            print("count error")
            return tau_scale, False
        
        # Abandon if tau_scale gets too small?
        if tau_scale*total_rate <= 1.0:
            print("scale error")
            return tau_scale, False
        
        count += 1

    return tau_scale, True

We provide the state of the system to prepare the function inputs:

In [4]:
x=np.array([100.,  1., 0.])    # (S=100, I=1, R=0)

reactant_mat=ode._lambdaMat  # Reactant_mat[i,j]=1 if state i involved in transition j and 0 if not

# Rates of transitions at current state x and time t
t=0  # No time dependence, so this is unimportant, just need to give something to function.
transition_func=ode.transition_vector
rates = transition_func(x, t)

First, there seems to be an issue with the reactant matrix

In [5]:
print(reactant_mat)

[[1 0 0 1 0 0]
 [1 1 0 0 1 0]
 [0 0 0 0 0 1]]


This correctly implies that states 0 and 1 ($S$ and $I$) are involved in infections (column 0).
It then incorrectly implies that only state 1 ($I$) is involved in recovery (column 1) and that no state is involved in births (column 2).
In the calculation of the reactant matrix, what is actually calculated is if the state appears as a variable in the equations.
i.e. $S$ and $I$ appear in infection rate $\frac{\beta S I}{N}$, but only $I$ appears in recovery rate $\gamma I$.

The second issue is that in the function `_test_tau_leap_safety`, we try to find the populations, xi, of the states involved in transition, i, via

```{python}
xi = x[reactant_mat[:, i]]
```

However, since reactant_mat only takes values 0 and 1, we are mistakenly recycling our binary indicators as state indices.
For example, let's look at the last transition, i=5, which should be a death rate from the recovereds:

In [6]:
x[reactant_mat[:, 5]]

array([100., 100.,   1.])

We should just get $[N_R]$.
Instead, we get $[N_S, N_S, N_I]$.

Perhaps we should instead use:

```{python}
xi = x[reactant_mat[:, i]==1]
```

In [7]:
x[reactant_mat[:, 5]==1]

array([0.])

Looks better, let's deifne a fixed version of the `_test_tau_leap_safety` function and compare them later:

In [8]:
def _test_tau_leap_safety_fix(x, reactant_mat, rates, tau_scale, epsilon):
    total_rate = sum(rates)  
    safe = False             # when True, indicates that tau_scale is sufficiently small
    count = 0                # number of attempts to find acceptable tau_scale

    # Print output, first some titles
    row=["Iteration", "tau_scale", "max_cdf"]
    print("{: <10} {: <10} {: <10}".format(*row))

    while safe is False:
        cdf_val = 1.0
        for i, r in enumerate(rates):
            xi = x[reactant_mat[:, i]==1]      # Population of each state involved in the transitions
            mu=tau_scale*r                     # Expected number of events
            new_cdf = _ppois(xi, mu=mu).min()  # prob(# transitions in jump < size of state)

            if new_cdf < cdf_val:
                cdf_val = new_cdf

        max_cdf = 1.0 - cdf_val  # prob(# transmissions in jump > size of state)

        # Print output
        row=[count, tau_scale, max_cdf]
        print("{: <10} {: <10.4g} {: <10.4g}".format(*row))

        # cannot allow it to exceed out epsilon
        if max_cdf > epsilon:
            tau_scale /= (max_cdf / epsilon)
        else:
            safe = True

        # Abandon if we're taking too many attempts
        if count > 256:
            print("count error")
            return tau_scale, False
        
        # Abandon if tau_scale gets too small?
        if tau_scale*total_rate <= 1.0:
            print("scale error")
            return tau_scale, False
        
        count += 1

    return tau_scale, True

We still have the issue that the `reactant_mat` which is input to the `_test_tau_leap_safety` function is not entirely correct.
For now, let's manually edit it and see the difference:

In [9]:
reactant_mat_fix=reactant_mat
reactant_mat_fix[2, 1]=1       # add the fact that R is involved in recovery
reactant_mat_fix[0, 2]=1       # add the fact that S is involved in birth
print(reactant_mat)

[[1 0 1 1 0 0]
 [1 1 0 0 1 0]
 [0 1 0 0 0 1]]


We need a value of the parameter epsilon, which dictates how stringent we are with the step size.
There is usually a preliminary calculation to obtain a first guess for `tau_scale`, for now we supply an overly large value so that the function has to do some work.

In [10]:
epsilon=0.1
tau_scale=10   # Set a silly high value, which algorithm should iteratively cut down

Let's compare the output of each function

In [11]:
y1=_test_tau_leap_safety_fix(x.astype(np.float64, copy=False),
                          reactant_mat_fix.astype(np.int64, copy=False),
                          rates.astype(np.float64, copy=False),
                          tau_scale=float(tau_scale),
                          epsilon=float(epsilon))

print(y1)
print("\n")

y2=_test_tau_leap_safety(x.astype(np.float64, copy=False),
                      reactant_mat.astype(np.int64, copy=False),
                      rates.astype(np.float64, copy=False),
                      tau_scale=float(tau_scale),
                      epsilon=float(epsilon))

print(y2)

Iteration  tau_scale  max_cdf   
0          10         1         
1          1          0.4734    
2          0.2112     0.05144   
(0.21122098761337782, True)


Iteration  tau_scale  max_cdf   
0          10         1         
1          1          1         
2          0.1        0.9995    
3          0.01       0.2644    
scale error
(0.003783681272942391, False)


We see that the old version fails due to the "scale error", namely

```{python}
tau_scale*total_rate <= 1.0:
```

It is worth pointing out that even if it didn't fail, it would not be returning correct values due to errors described above.

The next question is why is this scale error useful?
Is it that we don't want the step size getting so small that the probability of nothing happening becomes significant. 